In [71]:
from typing import Tuple, List
from datetime import datetime, timedelta, timezone
import pytz

import numpy as np

from sgp4.api import Satrec
from sgp4.api import jday
from astropy import coordinates
from astropy.coordinates.builtin_frames import GCRS, ITRS
import plotly.express as px
import plotly.graph_objects as go
import pymap3d as pm

from SATELLITES import SATELLITES
from coordinate_system.EarhRotating import build_default_rotating_cs
from XtoY.cartesianfromgeo import get_cartesian_geo_from_geo_angles
from coordinate_system.earth_models import get_wgs_84
from XtoY.xyz_rfilam import xyz_to_rfilam

from matplotlib import pyplot as plt

In [2]:
rotator = build_default_rotating_cs()
wgs84 = get_wgs_84() 

def get_satellite_trajectory(tle2lines: Tuple[str, str], time_points: List[datetime]):
    satellite = Satrec.twoline2rv(*tle2lines)
    jd_fr = np.array([jday(*(point.utctimetuple()[:6])) for point in time_points])
    result = [satellite.sgp4(jd, fr)[1] for jd, fr in jd_fr]
    return np.array(result)

def get_terrestrial_point_coordinates(geo_angles: Tuple[float, float], time_points: List[datetime]):
    coordinates = get_cartesian_geo_from_geo_angles(latitude=geo_angles[0], longitude=geo_angles[1], model=wgs84)
    time_points_timestamps = [time_point.timestamp() for time_point in time_points]
    rotation_matrix_list = rotator.get_matrix(time_points_timestamps)
    coordinates_in_inertial_system = np.array([rm.dot(coordinates) / 1000 for rm in rotation_matrix_list])
    return coordinates_in_inertial_system

def move_satellite_trajectory_to_earth_coordinates(
    trajectory: np.ndarray, time_points: List[datetime]
):
    time_points_timestamps = [time_point.timestamp() for time_point in time_points]
    rotation_matrix_list = rotator.get_matrix(time_points_timestamps)
    trajectory_in_earth_coordinates = np.array([rm.dot(coordinate) for rm, coordinate in zip(rotation_matrix_list, trajectory)])
    return trajectory_in_earth_coordinates

def get_related_spherical_coordinates(ref_point_coordinate, target_coordinates):
    dr_list = target_coordinates - ref_point_coordinate
#     print(dr_list)
    e1 = ref_point_coordinate
    e1 = e1 / np.linalg.norm(e1)
    e2 = np.cross([0, 0, 1], ref_point_coordinate)
    e2 = e2 / np.linalg.norm(e2)
    e3 = np.cross(e1, e2)
    e3 = e3 / np.linalg.norm(e3)
#     print(e1, e2, e3)

    coor_e1 = np.dot(dr_list, e1)
    coor_e2 = np.dot(dr_list, e2)
    coor_e3 = np.dot(dr_list, e3)
    length = np.linalg.norm(dr_list, axis=1)
#     print(coor_e1)
    
    lam = np.arcsin(coor_e3 / length) * 180 / np.pi
    phi = -np.arctan(coor_e2 / coor_e1) * 180 / np.pi
    return phi, lam, coor_e1



In [27]:
def ECEF_to_AER(r_sat, r_rec):
    '''Receive geocentric position of receiver and satelite respectively as numpy array. 
    Returns AER coordinates of satelite about receiver as numpy array'''
    #elipsoid coordinates
    x = r_rec[0]
    y = r_rec[1]
    z = r_rec[2]
    a = 6378137
    b = 6356752.3142
    e2 = 1 - (b/a)**2
    p = np.sqrt(x ** 2 + y ** 2)
    

    fi = np.arctan(z / ((1 - e2) * p))
    lam = np.arctan(y / x)
    
    #define local coordinate system (ENU) about the receiver
    e = np.array([-np.sin(lam),np.cos(lam),0])
    n = np.array([-np.cos(lam)*np.sin(fi), -np.sin(lam)*np.sin(fi), np.cos(fi)])
    u = np.array([np.cos(lam)*np.cos(fi), np.sin(lam)*np.cos(fi), np.sin(fi)])    
    
    #count azimuth elevation and range in system about receiver
    print("Shapes:", (r_sat - r_rec).shape, np.linalg.norm(r_sat - r_rec, axis=1).shape)
    po = (r_sat - r_rec) #/ (np.linalg.norm(r_sat - r_rec, axis=1))
    po = po / np.linalg.norm(po, axis=1)[:, np.newaxis]
    elevation = np.rad2deg(np.arcsin(po @ u))
    azimuth = np.rad2deg(np.arctan((po @ e) / (po @ n)))    
    rang = np.linalg.norm(r_sat - r_rec, axis=1)
    
    return np.array([azimuth, elevation, rang])

In [80]:
# t0 = datetime(2020, 9, 8, 14, 0, 0, tzinfo=timezone.utc)
t0 = datetime(2020, 9, 9, 11, 0, 0, tzinfo=timezone.utc)
time_points = [t0 + timedelta(seconds=i * 10)  for i in range(1600)]
print(time_points[-1])

lk_coord = [55.930148 / 180 * np.pi, 37.518151 / 180 * np.pi]

plots = []


for sat in SATELLITES:
    name = sat["name"]
    s = sat["s"]
    t = sat["t"]
    trajectory = get_satellite_trajectory((s, t), time_points)
    lk_coordinate = get_cartesian_geo_from_geo_angles(latitude=lk_coord[0], longitude=lk_coord[1], model=wgs84) / 1000
    earth_sk_trajectory = move_satellite_trajectory_to_earth_coordinates(trajectory, time_points)
#     phis, lams, coor_e1 = get_related_spherical_coordinates(lk_coordinate, earth_sk_trajectory)
    coord = pm.ecef2aer(earth_sk_trajectory[:, 0] * 1000, 
                        earth_sk_trajectory[:, 1] * 1000, 
                        earth_sk_trajectory[:, 2] * 1000, 
                        lk_coord[0], lk_coord[1], 130, deg=False)
    lams, phis, coor_e1 = coord
    lams = np.rad2deg(lams)
    phis = np.rad2deg(phis)
    phi=[]
    lam=[]
    times=[]
    
    for p, l, c, txt in zip(phis, lams, coor_e1, time_points):
#         if c > 0:
        if p > 0:
            phi.append(p)
            lam.append(l)
            times.append(txt.astimezone(pytz.timezone("Europe/Moscow")))
    plots.append(go.Scatterpolar(
            r = phi,
            theta = lam,
            text = times,
            name=name
        ))
    
fig = go.Figure(data=
    plots,
)
# fig.update_layout(
#     polar = dict(
#       radialaxis_range = [90, 0]
#     ))
fig.show()

2020-09-09 15:26:30+00:00
